In [1]:
import snowflake.connector
import pandas
import torch
from interactions import Interactions
from cross_validation import random_train_test_split

In [2]:
import snowflake.connector
import pandas as pd

def create_connection():
    ctx = snowflake.connector.connect(
        user='Zan',
        password='Kundalini55!',
        account='livenpay.ap-southeast-2')
    return ctx

def setup_snowflake(cursor):
    cursor.execute("use warehouse zans_wh")
    cursor.execute("use zans_db.postgres")
    
def query(cursor,query):
    cursor.execute(query)
    results = cursor.fetchall()
    return results

timestamped_sql = """
SELECT 
	accountcoupon_couponusehistory.time AS "accountcoupon_couponusehistory.time",
	account_localuser.id  AS "account_localuser.id",
	account_localuser.email  AS "account_localuser.email",
	merchant_merchant.name  AS "merchant_merchant.name",
	case when merchant_merchant.membership_zone_id = 1 then 'Melbourne'
    when merchant_merchant.membership_zone_id = 2 then 'Sydney'
    else null end AS "merchant_merchant.city",
	branch_branch.id  AS "branch_branch.id",
	branch_branch.suburb_id  AS "branch_branch.suburb_id"
FROM account_localuser AS account_localuser
LEFT JOIN accountcoupon_couponusehistory  AS accountcoupon_couponusehistory ON account_localuser.id = accountcoupon_couponusehistory.user_id 
LEFT JOIN branch_branch  AS branch_branch ON accountcoupon_couponusehistory.branch_id = branch_branch.id 
LEFT JOIN merchant_merchant  AS merchant_merchant ON merchant_merchant.id = branch_branch.merchant_id 

WHERE ((CASE WHEN account_localuser.is_staff  THEN 1 ELSE 0 END
) = 0) AND ((CASE
    WHEN DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())) is not null
    AND DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))) is not null /* date ranges or in the past x days */
    THEN
    CASE
    WHEN accountcoupon_couponusehistory.time >=  TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'This Period'
    WHEN accountcoupon_couponusehistory.time >=
    DATEADD('day',
      DATEDIFF('day',
        TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))),
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )*-1,
      DATEADD('day', -1,
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )
    )
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('day', 1,DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'Previous Period'
    END
    END
) IS NOT NULL
)
GROUP BY 7,1,2,3,4,5,6
ORDER BY 1 DESC
"""

sql = """
SELECT 
	account_localuser.id  AS "account_localuser.id",
	account_localuser.email  AS "account_localuser.email",
	merchant_merchant.name  AS "merchant_merchant.name",
	case when merchant_merchant.membership_zone_id = 1 then 'Melbourne'
    when merchant_merchant.membership_zone_id = 2 then 'Sydney'
    else null end AS "merchant_merchant.city",
	branch_branch.id  AS "branch_branch.id",
	branch_branch.suburb_id  AS "branch_branch.suburb_id"
FROM account_localuser AS account_localuser
LEFT JOIN accountcoupon_couponusehistory  AS accountcoupon_couponusehistory ON account_localuser.id = accountcoupon_couponusehistory.user_id 
LEFT JOIN branch_branch  AS branch_branch ON accountcoupon_couponusehistory.branch_id = branch_branch.id 
LEFT JOIN merchant_merchant  AS merchant_merchant ON merchant_merchant.id = branch_branch.merchant_id 

WHERE ((CASE WHEN account_localuser.is_staff  THEN 1 ELSE 0 END
) = 0) AND ((CASE
    WHEN DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())) is not null
    AND DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))) is not null /* date ranges or in the past x days */
    THEN
    CASE
    WHEN accountcoupon_couponusehistory.time >=  TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'This Period'
    WHEN accountcoupon_couponusehistory.time >=
    DATEADD('day',
      DATEDIFF('day',
        TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))),
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )*-1,
      DATEADD('day', -1,
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )
    )
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('day', 1,DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'Previous Period'
    END
    END
) IS NOT NULL
)
GROUP BY 1,2,3,4,5,6
ORDER BY 1 DESC
"""

ctx = create_connection()
cursor = ctx.cursor()
setup_snowflake(cursor)

TRANSACTIONS = query(cursor,sql)

cursor.close()
ctx.close()

all_data = pd.DataFrame(TRANSACTIONS, columns=['userId','email','merchant','city','branchId','suburb'])
#all_data = pd.DataFrame(TRANSACTIONS, columns=['time','userId','email','merchant','city','branchId','suburb'])

In [3]:
CITY = "Sydney"

cull = ['Uber','Secure Parking Melbourne','Taste Of Melbourne','UberEATS Sydney','Liven Bistro'
       , 'Cardly - You Write, We Post', 'Good Food & Wine Show', 'Sayers Sister [Kounta Demo]'
       , 'Airtasker','Peninsula Hot Springs','Melbourne River Cruises','Grilla House','Margaret River Gourmet Escape','Madeira','Royale Fusion'
        ,'The Bar.Ber','Tandoori Point','The Italian Cucina','Tokyo Bar','The Trust'
        ,"Orita's",'Pelican ','Olive Garden','Golftec','Lash Labs','Cafe on Bourke','Ibuki House','Mint Leaf',"Tina's Noodle Kitchen"
       ,'Uber Sydney','Cardly - You Write, We Post','Secure Parking Sydney','UberEATS Sydney','Liven Bistro','Good Food & Wine Show','Sayers Sister [Kounta Demo]','Airtasker','Taste Of Sydney']

for merchant in cull:
    all_data = all_data[all_data.merchant!=merchant]
    
all_data = all_data.dropna()
all_data.branchId = all_data.branchId.astype(int)
city_data = all_data[all_data.city==CITY]
city_data.head()

,userId,email,merchant,city,branchId,suburb
3,366067,bdm@rnjrealty.com.au,Ajisen Ramen,Sydney,1482,411.0
4,366064,preciousngo@hotmail.com,CoCo Fresh Tea & Juice,Sydney,1246,418.0
13,366045,elizamcgowan@icloud.com,Messina,Sydney,1113,411.0
14,366044,ryan@lonstein.net,BL Burgers,Sydney,1494,422.0
15,366041,jess.lee.phillips@live.com.au,Oh! Matcha,Sydney,1459,411.0


In [4]:
userId,branchId,suburb = city_data['userId'],city_data['branchId'],city_data['suburb']
interactions = pd.DataFrame({"userId":userId,"branchId":branchId,"suburb":suburb})
interactions = interactions[interactions.groupby('userId').userId.transform(len) > 1]

interactions.head()

,userId,branchId,suburb
66,365916,1247,411.0
67,365916,1458,411.0
68,365914,1115,489.0
69,365914,1575,411.0
104,365427,1242,411.0


In [5]:
from collections import defaultdict

userIds = interactions.userId.unique()
itemIds = interactions.branchId.unique()
internal_userIds = defaultdict(lambda: len(internal_userIds))
internal_itemIds = defaultdict(lambda: len(internal_itemIds))

external_userIds = {}
external_itemIds = {}

for user,item in zip(interactions.userId.values,interactions.branchId.values):
    internal_userIds[user]
    internal_itemIds[item]
    external_userIds[internal_userIds[user]] = user
    external_itemIds[internal_itemIds[item]] = item

In [6]:
interactions.head()

,userId,branchId,suburb
66,365916,1247,411.0
67,365916,1458,411.0
68,365914,1115,489.0
69,365914,1575,411.0
104,365427,1242,411.0


In [7]:
interactions_proc = interactions

internalItems = []
internalUsers = []

for i,r in interactions.iterrows():
    internalItems += [internal_itemIds[int(r.branchId)]]
    internalUsers += [internal_userIds[int(r.userId)]]
    
interactions_proc['branchId'] = internalItems
interactions_proc['userId'] = internalUsers

In [8]:
interactions_proc.loc[interactions_proc['userId']==1]

,userId,branchId,suburb
68,1,2,489.0
69,1,3,411.0


In [9]:
userIds = interactions_proc.userId.values
itemIds = interactions_proc.branchId.values
suburbs = interactions_proc.suburb.values

In [10]:
interactions_proc.head()

,userId,branchId,suburb
66,0,0,411.0
67,0,1,411.0
68,1,2,489.0
69,1,3,411.0
104,2,4,411.0


In [11]:
featureDict = {}

for i in itemIds:
    featureDict[i] = int(interactions_proc.loc[interactions_proc['branchId']==i]['suburb'].unique())

In [12]:
item_features = list(featureDict.values())

In [13]:
type(userIds)

numpy.ndarray

In [14]:
import numpy as np
feature_interactions = Interactions(userIds,itemIds,item_features=np.asarray(item_features))

In [15]:
feature_interactions

<Interactions dataset (13541 users x 314 items x 51182 interactions)>

In [16]:
feature_interactions.item_features

array([  411.,   411.,   489.,   411.,   411.,   421.,   411.,   418.,
         411.,   411.,   411.,   411.,   408.,   408.,   408.,   411.,
         411.,   408.,   418.,   408.,   408.,   411.,   411.,   408.,
         426.,   411.,   547.,   411.,   411.,   677.,   701.,   411.,
         411.,   422.,   411.,   411.,   411.,   408.,   411.,   411.,
         422.,   411.,   411.,   411.,   411.,   411.,   453.,   411.,
         411.,   526.,   547.,   534.,   411.,   499.,   547.,   678.,
         718.,   419.,   419.,   521.,   423.,   408., 16082.,   772.,
         419.,   411.,   408.,   772.,   547.,   419.,   419.,   418.,
         521.,   411.,   474.,   419.,   419.,   470.,   499.,   411.,
         411.,   434.,   408.,   711.,   435.,   892.,   411., 16082.,
         411.,   411.,   408.,   411.,   411.,   411.,   411.,   436.,
         417.,   411.,   411.,   423.,   411.,   470.,   411.,   446.,
         411.,   411.,   411.,   958.,   836.,   411.,   892.,   673.,
      

In [21]:
"""
Factorization models for implicit feedback problems.
"""

import numpy as np

import torch

import torch.optim as optim

from torch.autograd import Variable

from helpers import _repr_model
from factorization._components import (_predict_process_features,
                                                 _predict_process_ids)
from losses import (adaptive_hinge_loss,
                              bpr_loss,
                              hinge_loss,
                              pointwise_loss)

from factorization.representations import (BilinearNet,
                                                     FeatureNet,
                                                     HybridContainer)

from sampling import sample_items
from torch_utils import cpu, gpu, set_seed


class ImplicitFactorizationModel(object):

    def __init__(self,
                 loss='pointwise',
                 embedding_dim=32,
                 n_iter=10,
                 batch_size=256,
                 l2=0.0,
                 learning_rate=1e-2,
                 optimizer_func=None,
                 use_cuda=False,
                 sparse=False,
                 random_state=None,
                 representation=None,
                 n_components=2):

        assert loss in ('pointwise',
                        'bpr',
                        'hinge',
                        'adaptive_hinge')

        self._loss = loss
        self._embedding_dim = embedding_dim
        self._n_iter = n_iter
        self._learning_rate = learning_rate
        self._batch_size = batch_size
        self._l2 = l2
        self._use_cuda = use_cuda
        self._sparse = sparse
        self._optimizer_func = optimizer_func
        self._random_state = random_state or np.random.RandomState()

        self._num_users = None
        self._num_items = None
        self._net = None
        self._optimizer = None
        self._loss_func = None
        self._representation=representation
        self._num_components=n_components

        set_seed(self._random_state.randint(-10**8, 10**8),
                 cuda=self._use_cuda)

    def __repr__(self):

        return _repr_model(self)

    @property
    def _initialized(self):
        return self._net is not None

    def _initialize(self, interactions):

        (self._num_users,
         self._num_items) = (interactions.num_users,
                             interactions.num_items)
        
        if self._representation=='mixture':
            latent_net = MixtureNet(self._num_users,
                                   self._num_items,
                                    self._embedding_dim,
                                   num_components=self._num_components)
        
        elif self._representation=='nonlinear_mixture':
            latent_net = NonlinearMixtureNet(self._num_users,
                                             self._num_items,
                                             self._embedding_dim)
            
        elif self._representation=='embedding_mixture':
            latent_net = EmbeddingMixtureNet(self._num_users,
                                            self._num_items,
                                            self._embedding_dim)

        else:
            latent_net = BilinearNet(self._num_users,
                                     self._num_items,
                                     self._embedding_dim,
                                     sparse=self._sparse)
            
            

        if interactions.num_user_features():
            user_net = FeatureNet(interactions.num_user_features(),
                                  self._embedding_dim)
        else:
            user_net = None

        if interactions.num_context_features():
            context_net = FeatureNet(interactions.num_context_features(),
                                     self._embedding_dim)
        else:
            context_net = None

        if interactions.num_item_features():
            item_net = FeatureNet(interactions.num_item_features(),
                                  self._embedding_dim)
        else:
            item_net = None

        self._net = gpu(HybridContainer(latent_net,
                                        user_net,
                                        context_net,
                                        item_net),
                        self._use_cuda)

        if self._optimizer_func is None:
            self._optimizer = optim.Adam(
                self._net.parameters(),
                weight_decay=self._l2,
                lr=self._learning_rate
            )
        else:
            self._optimizer = self._optimizer_func(self._net.parameters())

        if self._loss == 'pointwise':
            self._loss_func = pointwise_loss
        elif self._loss == 'bpr':
            self._loss_func = bpr_loss
        elif self._loss == 'hinge':
            self._loss_func = hinge_loss
        else:
            self._loss_func = adaptive_hinge_loss

    def _check_input(self, user_ids, item_ids, allow_items_none=False):

        if isinstance(user_ids, int):
            user_id_max = user_ids
        else:
            user_id_max = user_ids.max()

        if user_id_max >= self._num_users:
            raise ValueError('Maximum user id greater '
                             'than number of users in model.')

        if allow_items_none and item_ids is None:
            return

        if isinstance(item_ids, int):
            item_id_max = item_ids
        else:
            item_id_max = item_ids.max()

        if item_id_max >= self._num_items:
            raise ValueError('Maximum item id greater '
                             'than number of items in model.')

    def fit(self, interactions, verbose=False):

        user_ids = interactions.user_ids.astype(np.int64)
        item_ids = interactions.item_ids.astype(np.int64)

        if not self._initialized:
            self._initialize(interactions)

        self._check_input(user_ids, item_ids)

        for epoch_num in range(self._n_iter):

            interactions.shuffle(random_state=self._random_state)

            epoch_loss = 0.0

            for (minibatch_num,
                 minibatch) in enumerate(interactions
                                         .minibatches(use_cuda=self._use_cuda,
                                                      batch_size=self._batch_size)):

                minibatch = minibatch.torch(self._use_cuda).variable()

                positive_prediction = self._net(minibatch.user_ids,
                                                minibatch.item_ids,
                                                minibatch.user_features,
                                                minibatch.context_features,
                                                minibatch.item_features)
                
                if self._loss == 'adaptive_hinge':
                    negative_prediction = [self._get_negative_prediction(minibatch)
                                           for _ in range(5)]
                else:
                    negative_prediction = self._get_negative_prediction(minibatch)

                self._optimizer.zero_grad()

                loss = self._loss_func(
                    positive_prediction,
                    negative_prediction,
                    weights=minibatch.weights
                )

                epoch_loss += loss.data[0]

                loss.backward()
                self._optimizer.step()

            epoch_loss /= minibatch_num + 1

            if verbose:
                print('Epoch {}: loss {}'.format(epoch_num, epoch_loss))

    def _get_negative_prediction(self, minibatch):

        negative_items = sample_items(
            self._num_items,
            len(minibatch),
            random_state=self._random_state)
        
        negative_var = gpu(torch.from_numpy(negative_items), self._use_cuda)

        negative_prediction = self._net(minibatch.user_ids,
                                        negative_var,
                                        minibatch.user_features,
                                        minibatch.context_features,
                                        minibatch.item_features)

        return negative_prediction
    
    def predict(self, user_ids, item_ids=None,
            user_features=None,
            context_features=None,
            item_features=None):


        self._check_input(user_ids, item_ids, allow_items_none=True)
        self._net.train(False)

        user_ids, item_ids = _predict_process_ids(user_ids, item_ids,
                                                  self._num_items,
                                                  self._use_cuda)

        (user_features,
         context_features,
         item_features) = _predict_process_features(user_features,
                                                    context_features,
                                                    item_features,
                                                    len(item_ids),
                                                    self._use_cuda)

        out = self._net(user_ids,
                        item_ids,
                        user_features,
                        context_features,
                        item_features)

        return cpu(out.data).numpy().flatten()

### Hybrid containers, feature nets, and mixture representations

In [22]:
import random

import torch

import torch.nn as nn
import torch.nn.init
import torch.nn.functional as F

from torch.autograd import Variable

from spotlight.layers import ScaledEmbedding, ZeroEmbedding

"""
Classes defining user and item latent representations in
factorization models.
"""

import torch.nn as nn
import torch.nn.functional as F

from spotlight.layers import ScaledEmbedding, ZeroEmbedding


class HybridContainer(nn.Module):

    def __init__(self,
                 latent_module,
                 user_module=None,
                 context_module=None,
                 item_module=None):

        super(HybridContainer, self).__init__()

        self.latent = latent_module
        self.user = user_module
        self.context = context_module
        self.item = item_module

    def forward(self, user_ids,
                item_ids,
                user_features=None,
                context_features=None,
                item_features=None):

        user_representation, user_bias = self.latent.user_representation(user_ids)
        item_representation, item_bias = self.latent.item_representation(item_ids)

        if self.user is not None:
            user_representation += self.user(user_features)
        if self.context is not None:
            user_representation += self.context(context_features)
        if self.item is not None:
            item_representation += self.item(item_features)

        dot = (user_representation * item_representation).sum(1)

        return dot + user_bias + item_bias


class FeatureNet(nn.Module):

    def __init__(self, input_dim, output_dim, bias=False, nonlinearity='tanh'):

        super(FeatureNet, self).__init__()

        if nonlinearity == 'tanh':
            self.nonlinearity = F.tanh
        elif nonlinearity == 'relu':
            self.nonlinearity = F.relu
        elif nonlinearity == 'sigmoid':
            self.nonlinearity = F.sigmoid
        elif nonlinearity == 'linear':
            self.nonlinearity = lambda x: x
        else:
            raise ValueError('Nonlineariy must be one of '
                             '(tanh, relu, sigmoid, linear)')

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.fc_1 = nn.Linear(self.input_dim,
                              self.output_dim,
                              bias=bias)

    def forward(self, features):

        return self.nonlinearity(self.fc_1(features))


class BilinearNet(nn.Module):
    """
    Bilinear factorization representation.
    Encodes both users and items as an embedding layer; the score
    for a user-item pair is given by the dot product of the item
    and user latent vectors.
    Parameters
    ----------
    num_users: int
        Number of users in the model.
    num_items: int
        Number of items in the model.
    embedding_dim: int, optional
        Dimensionality of the latent representations.
    sparse: boolean, optional
        Use sparse gradients.
    """

    def __init__(self, num_users, num_items, embedding_dim=32, sparse=False):

        super(BilinearNet, self).__init__()

        self.embedding_dim = embedding_dim

        self.user_embeddings = ScaledEmbedding(num_users, embedding_dim,
                                               sparse=sparse)
        self.item_embeddings = ScaledEmbedding(num_items, embedding_dim,
                                               sparse=sparse)
        self.user_biases = ZeroEmbedding(num_users, 1, sparse=sparse)
        self.item_biases = ZeroEmbedding(num_items, 1, sparse=sparse)

    def user_representation(self, user_ids):

        user_embedding = self.user_embeddings(user_ids)
        user_embedding = user_embedding.view(-1, self.embedding_dim)

        user_bias = self.user_biases(user_ids).view(-1, 1)

        return user_embedding, user_bias

    def item_representation(self, item_ids):

        item_embedding = self.item_embeddings(item_ids)
        item_embedding = item_embedding.view(-1, self.embedding_dim)

        item_bias = self.item_biases(item_ids).view(-1, 1)

        return item_embedding, item_bias

    def forward(self, user_representation, user_bias, item_representation, item_bias):
        """
        Compute the forward pass of the representation.
        Parameters
        ----------
        user_ids: tensor
            Tensor of user indices.
        item_ids: tensor
            Tensor of item indices.
        Returns
        -------
        predictions: tensor
            Tensor of predictions.
        """

        dot = (user_representation * item_representation).sum(1)

        return dot + user_bias + item_bias

class MixtureNet(nn.Module):
    """
    Bilinear factorization representation.
    Encodes both users and items as an embedding layer; the score
    for a user-item pair is given by the dot product of the item
    and user latent vectors.
    Parameters
    ----------
    num_users: int
        Number of users in the model.
    num_items: int
        Number of items in the model.
    embedding_dim: int, optional
        Dimensionality of the latent representations.
    user_embedding_layer: an embedding layer, optional
        If supplied, will be used as the user embedding layer
        of the network.
    item_embedding_layer: an embedding layer, optional
        If supplied, will be used as the item embedding layer
        of the network.
    sparse: boolean, optional
        Use sparse gradients.
    """

    def __init__(self, num_users, num_items, embedding_dim=32,
                 projection_scale=1.0,
                 num_components=4):

        super(MixtureNet, self).__init__()

        self.embedding_dim = embedding_dim
        self.num_components = num_components
        self.projection_scale = projection_scale

        self.user_embeddings = ScaledEmbedding(num_users, embedding_dim)
        self.item_embeddings = ScaledEmbedding(num_items, embedding_dim)

        self.user_biases = ZeroEmbedding(num_users, 1)
        self.item_biases = ZeroEmbedding(num_items, 1)

        self.taste_projection = nn.Linear(embedding_dim,
                                          embedding_dim * self.num_components, bias=False)
        self.attention_projection = nn.Linear(embedding_dim,
                                              embedding_dim * self.num_components, bias=False)

        for layer in (self.taste_projection, self.attention_projection):
            torch.nn.init.xavier_normal(layer.weight, self.projection_scale)

    def user_representation(self, user_ids):

        user_embedding = self.user_embeddings(user_ids).squeeze()
        user_bias = self.user_biases(user_ids).squeeze()

        return user_embedding, user_bias

    def item_representation(self, item_ids):

        item_embedding = self.item_embeddings(item_ids).squeeze()
        item_bias = self.item_biases(item_ids).squeeze()

        return item_embedding, item_bias

    def forward(self, user_ids, item_ids):

        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        batch_size, embedding_size = item_embedding.size()

        user_tastes = (self.taste_projection(user_embedding)
                       .resize(batch_size,
                               self.num_components,
                               embedding_size))
        user_attention = (self.attention_projection(user_embedding)
                          .resize(batch_size,
                                  self.num_components,
                                  embedding_size))
        user_attention = user_attention #  * user_embedding.unsqueeze(1).expand_as(user_tastes)

        attention = (F.softmax((user_attention *
                                item_embedding.unsqueeze(1).expand_as(user_attention))
                               .sum(2)).unsqueeze(2).expand_as(user_attention))
        weighted_preference = (user_tastes * attention).sum(1)

        dot = (weighted_preference * item_embedding).sum(1)

        user_bias = self.user_biases(user_ids).squeeze()
        item_bias = self.item_biases(item_ids).squeeze()

        if random.random() < 0.5:
            print('User tastes', user_tastes[0][0].abs().mean().data[0])
            print('Tastes', weighted_preference.abs().mean().data[0])
        #     print('Attention', (user_attention * item_embedding).sum(2).max().data[0])
        #     print('Softmax', attention.max(1)[0].mean().data[0])
            print('Preference', preference.max(1)[0].mean().data[0])
        #     print('Prediction', weighted_preference.mean().data[0])
        #     print('Biases', user_bias.max().data[0], item_bias.max().data[0])

        return dot + user_bias + item_bias


class MixtureComponent(nn.Module):

    def __init__(self, embedding_dim, num_components):

        super(MixtureComponent, self).__init__()

        self.embedding_dim = embedding_dim
        self.num_components = num_components

        self.fc_1 = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.fc_2 = nn.Linear(embedding_dim, embedding_dim, bias=False)

        self.taste_projection = nn.Linear(embedding_dim,
                                          embedding_dim * num_components,
                                          bias=False)
        self.attention_projection = nn.Linear(embedding_dim,
                                              embedding_dim * num_components,
                                              bias=False)

    def forward(self, x):

        batch_size, embedding_size = x.size()

        x = F.relu(self.fc_1(x))
        x = F.relu(self.fc_2(x))

        user_tastes = (self.taste_projection(x)
                       .resize(batch_size,
                               self.num_components,
                               embedding_size))
        user_attention = (self.attention_projection(x)
                          .resize(batch_size,
                                  self.num_components,
                                  embedding_size))

        return user_tastes, user_attention


class NonlinearMixtureNet(nn.Module):
    """
    Bilinear factorization representation.
    Encodes both users and items as an embedding layer; the score
    for a user-item pair is given by the dot product of the item
    and user latent vectors.
    Parameters
    ----------
    num_users: int
        Number of users in the model.
    num_items: int
        Number of items in the model.
    embedding_dim: int, optional
        Dimensionality of the latent representations.
    user_embedding_layer: an embedding layer, optional
        If supplied, will be used as the user embedding layer
        of the network.
    item_embedding_layer: an embedding layer, optional
        If supplied, will be used as the item embedding layer
        of the network.
    sparse: boolean, optional
        Use sparse gradients.
    """

    def __init__(self, num_users, num_items, embedding_dim=32,
                 num_components=4):

        super(NonlinearMixtureNet, self).__init__()

        self.embedding_dim = embedding_dim
        self.num_components = num_components

        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim)

        self.user_biases = ZeroEmbedding(num_users, 1)
        self.item_biases = ZeroEmbedding(num_items, 1)

        self.mixture = MixtureComponent(embedding_dim, num_components)
        
    def user_representation(self, user_ids):

        user_embedding = self.user_embeddings(user_ids).squeeze()
        user_bias = self.user_biases(user_ids).squeeze()

        return user_embedding, user_bias

    def item_representation(self, item_ids):

        item_embedding = self.item_embeddings(item_ids).squeeze()
        item_bias = self.item_biases(item_ids).squeeze()

        return item_embedding, item_bias

    def forward(self, user_ids, item_ids):
        """
        Compute the forward pass of the representation.
        Parameters
        ----------
        user_ids: tensor
            Tensor of user indices.
        item_ids: tensor
            Tensor of item indices.
        Returns
        -------
        predictions: tensor
            Tensor of predictions.
        """

        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        batch_size, embedding_size = item_embedding.size()

        user_tastes, user_attention = self.mixture(user_embedding)
        item_embedding = item_embedding.unsqueeze(1).expand_as(user_attention)

        attention = F.softmax((user_attention * item_embedding).sum(2))

        preference = ((user_tastes * item_embedding)
                      .sum(2))
        weighted_preference = (attention * preference).sum(1).squeeze()

        user_bias = self.user_biases(user_ids).squeeze()
        item_bias = self.item_biases(item_ids).squeeze()

        return weighted_preference + user_bias + item_bias

In [119]:
train,test = random_train_test_split(feature_interactions)

n_tastes = 2

model = ImplicitFactorizationModel(n_iter=10,
                                   loss='hinge',
                                   l2=-1.252542352e-10,
                                   representation='mixture',
                                  n_components=n_tastes)
model.fit(train,verbose=True)


from evaluation import mrr_score
mrr = mrr_score(model, test)
print('mean mrr',mrr.mean())

/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:204: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:218: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 0: loss 0.5734049677848816
Epoch 1: loss 0.28838491439819336
Epoch 2: loss 0.16966567933559418
Epoch 3: loss 0.12763722240924835
Epoch 4: loss 0.10339103639125824
Epoch 5: loss 0.09451333433389664
Epoch 6: loss 0.08411861211061478
Epoch 7: loss 0.0812554657459259
Epoch 8: loss 0.076598159968853
Epoch 9: loss 0.07150653749704361
mean mrr 0.12720720031353022


In [121]:
model

<ImplicitFactorizationModel: HybridContainer(
  (latent): MixtureNet(
    (user_embeddings): ScaledEmbedding(13541, 32)
    (item_embeddings): ScaledEmbedding(314, 32)
    (user_biases): ZeroEmbedding(13541, 1)
    (item_biases): ZeroEmbedding(314, 1)
    (taste_projection): Linear(in_features=32, out_features=64, bias=False)
    (attention_projection): Linear(in_features=32, out_features=64, bias=False)
  )
  (item): FeatureNet(
    (fc_1): Linear(in_features=314, out_features=32, bias=False)
  )
)>

In [122]:
for i in model._net.parameters():
    print(i[:1])

tensor([[-0.2818,  0.2358, -0.1447,  0.1530,  0.2601, -0.1392, -0.0484,
         -0.1368,  0.2521,  0.1030, -0.1966, -0.2727,  0.1691,  0.1387,
         -0.2577,  0.1317, -0.1030,  0.2760,  0.1266,  0.2096,  0.2248,
         -0.1078,  0.1155,  0.2455,  0.2426, -0.1097,  0.2501, -0.1546,
          0.2178,  0.1011,  0.1746,  0.2147]])
tensor([[ 0.1846,  0.3101,  0.0900, -0.2711,  0.9544,  0.7490, -0.6171,
         -0.1845,  0.1995, -0.4385, -0.6132, -0.8229,  0.1232,  0.1719,
          0.0048, -0.5014,  0.1268,  0.4213, -0.4588,  0.1314, -0.2703,
         -0.0272, -0.1857, -0.3243, -0.2580, -0.3034,  0.0146, -0.3015,
          0.5201, -0.9172, -0.2361, -0.0938]])
tensor([[ 0.]])
tensor([[ 2.1021]])
tensor([[-0.2309,  0.1621, -0.0111,  0.1826,  0.2127,  0.0141, -0.0356,
          0.1589,  0.1389,  0.2627, -0.1116, -0.1009,  0.1247, -0.1725,
         -0.1611,  0.0831,  0.0388, -0.0961,  0.0343,  0.0122,  0.0945,
          0.2049,  0.0966, -0.1781,  0.4067, -0.0738, -0.1114,  0.1192,
      

In [123]:
learned = list(model._net.parameters())

user_embeddings = learned[0]
item_embeddings = learned[1]

user_embeddings = list(zip(user_embeddings,range(0,feature_interactions.num_users)))
item_embeddings = list(zip(item_embeddings,range(0,feature_interactions.num_items)))


In [124]:
external_userIds[13495]

39993

In [125]:
user_dict = {}

for emb,u in user_embeddings:
    try:
        user_dict[external_userIds[u]] = emb
    except:
        print(u)

In [126]:
item_dict = {}

for emb,i in item_embeddings:
    try:
        item_dict[external_itemIds[i]] = emb
    except:
        print(u)

In [127]:
fnc = lambda x:x[0]

fnc(item_embeddings)

(tensor([ 0.1846,  0.3101,  0.0900, -0.2711,  0.9544,  0.7490, -0.6171,
         -0.1845,  0.1995, -0.4385, -0.6132, -0.8229,  0.1232,  0.1719,
          0.0048, -0.5014,  0.1268,  0.4213, -0.4588,  0.1314, -0.2703,
         -0.0272, -0.1857, -0.3243, -0.2580, -0.3034,  0.0146, -0.3015,
          0.5201, -0.9172, -0.2361, -0.0938]), 0)

In [166]:
internalId=2
userId=external_userIds[internalId]

user = list(zip(range(feature_interactions.num_items),[user_dict[userId].dot(emb) for emb in learned[1]]))

In [167]:
preds = sorted(user,key=lambda x:x[1],reverse=True)

In [168]:
city_data[city_data['userId']==userId]

,userId,email,merchant,city,branchId,suburb
104,365427,liqyx@yahoo.com.tw,CoCo Fresh Tea & Juice,Sydney,1242,411.0
105,365427,liqyx@yahoo.com.tw,Din Tai Fung,Sydney,1538,421.0
106,365427,liqyx@yahoo.com.tw,Tontaro Ramen,Sydney,1588,411.0


In [169]:
preds = sorted(user,key=lambda x:x[1],reverse=True)

for i in range(20):
    branchId = external_itemIds[preds[i][0]]
    x = city_data[city_data['branchId']==branchId]
    merchant = x.merchant.unique()
    suburb = x.suburb.unique()
    print(preds[i][1],' ------ ',merchant[0],': ',str(suburb[0]))
    

tensor(3.9755)  ------  Tontaro Ramen :  411.0
tensor(3.7341)  ------  Din Tai Fung :  421.0
tensor(3.6773)  ------  Din Tai Fung :  958.0
tensor(3.0674)  ------  Eat Fuh :  892.0
tensor(2.9645)  ------  YiFang Taiwan Fruit Tea :  411.0
tensor(2.8739)  ------  TNS Coffee Project :  419.0
tensor(2.8434)  ------  CoCo Fresh Tea & Juice :  411.0
tensor(2.8253)  ------  CoCo Fresh Tea & Juice :  408.0
tensor(2.8113)  ------  Din Tai Fung :  526.0
tensor(2.8028)  ------  Sharetea :  411.0
tensor(2.7719)  ------  Meet Fresh :  408.0
tensor(2.6199)  ------  CoCo Fresh Tea & Juice :  418.0
tensor(2.6175)  ------  Bubble Nini Tea :  419.0
tensor(2.4962)  ------  Ajisen Ramen :  411.0
tensor(2.4537)  ------  CoCo Fresh Tea & Juice :  411.0
tensor(2.4278)  ------  Enjoy Cafe by Enjoy Mie :  411.0
tensor(2.4030)  ------  CoCo Fresh Tea & Juice :  435.0
tensor(2.1336)  ------  CoCo Fresh Tea & Juice :  418.0
tensor(2.1095)  ------  Oh! Matcha :  411.0
tensor(2.0526)  ------  SumoSalad :  716.0


In [170]:
taste_projection = nn.Linear(32,32 * n_tastes, bias=False)
print(learned[4])
taste_projection.weight = learned[4]
print(taste_projection.weight==learned[4])


Parameter containing:
tensor([[-0.2309,  0.1621, -0.0111,  ...,  0.1552, -0.0726,  0.2223],
        [-0.1488, -0.1038, -0.2354,  ...,  0.1795, -0.2043, -0.2836],
        [ 0.0858,  0.1737, -0.0180,  ...,  0.1713,  0.2585, -0.0894],
        ...,
        [-0.0351,  0.1418,  0.1889,  ...,  0.3052, -0.0912,  0.2476],
        [-0.0448, -0.1151,  0.1705,  ...,  0.0301, -0.0131,  0.0723],
        [ 0.0954,  0.0016,  0.0554,  ...,  0.1327,  0.1234,  0.0061]])
tensor([[ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1],
        ...,
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1]], dtype=torch.uint8)


In [171]:
tastes = taste_projection(user_dict[userId]).resize(n_tastes,32)

In [172]:
tastes

tensor([[ 0.1257, -0.6785,  0.4028,  0.6221,  0.1428, -0.0869, -0.2074,
         -0.0290,  0.1835,  0.5776,  0.1427,  0.6573, -0.2319, -0.0322,
          0.4825,  0.3988, -0.4860,  0.0883,  0.8176,  0.3656,  0.2747,
          0.1157, -0.3219, -0.2323, -0.2095, -0.4324,  0.2579, -0.3439,
          0.5007, -0.3284,  0.4328, -0.1140],
        [ 0.1175,  0.2679,  0.1728, -0.1262,  0.7739,  0.5620, -0.4101,
         -0.0028,  0.0390,  0.0554, -0.3447,  0.3714, -0.1323,  0.1399,
         -0.0456, -0.7780,  0.3907, -0.3821, -0.5116, -0.0446, -0.1088,
         -0.6998,  0.0186, -0.0162, -0.5486,  0.0920, -0.3349,  0.0504,
          0.0360, -0.7610,  0.2625, -0.3747]])

In [173]:
preds = []
for taste in tastes:
    preds+=[list(zip(range(feature_interactions.num_items),[taste.dot(emb) for emb in learned[1]]))]

In [174]:
count = 0
for recs in preds:
    recs = sorted(recs,key=lambda x:x[1],reverse=True)
    count += 1
    print('Taste profile: ',count)
    for i in range(10):
        branchId = external_itemIds[recs[i][0]]
        x = city_data[city_data['branchId']==branchId]
        merchant = x.merchant.unique()
        suburb = x.suburb.unique()
        print(recs[i][1],' ------ ',merchant[0],': ',str(suburb[0]))
    print('\n')


Taste profile:  1
tensor(3.3222)  ------  Two Sticks :  16082.0
tensor(3.1065)  ------  Mr Bao :  411.0
tensor(3.0644)  ------  PappaRich :  411.0
tensor(2.7762)  ------  YiFang Taiwan Fruit Tea :  521.0
tensor(2.7562)  ------  New Shanghai :  547.0
tensor(2.7202)  ------  La Lucha :  411.0
tensor(2.6394)  ------  Nama :  411.0
tensor(2.6201)  ------  Treetop Cafe :  435.0
tensor(2.5574)  ------  85 Degrees Cafe :  411.0
tensor(2.4985)  ------  Naya Cafe :  490.0


Taste profile:  2
tensor(3.6199)  ------  Sharetea :  677.0
tensor(2.8677)  ------  Sexy Tea :  408.0
tensor(2.8057)  ------  CoCo Fresh Tea & Juice :  411.0
tensor(2.7598)  ------  Pasta Pantry :  411.0
tensor(2.7013)  ------  Sushi Rap :  411.0
tensor(2.4968)  ------  Shinsen Sushi :  411.0
tensor(2.3899)  ------  Aqua S :  411.0
tensor(2.2000)  ------  Sharetea :  474.0
tensor(2.1627)  ------  Ho Jiak :  718.0
tensor(2.0887)  ------  Oh! Matcha :  411.0


